<a href="https://colab.research.google.com/github/Cflalex/CflAlex95/blob/main/%D0%A1%D0%B8%D0%BD%D1%82%D0%B5%D1%82%D0%B8%D0%BA%D0%B0_%D0%9F%D0%B0%D1%80%D0%BD%D0%B8%D0%BA_%D0%921.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime

# Количество строк в файле
n_rows = 288  # Данные каждые 5 минут на протяжении 24 часов

# Инициализация времени и даты
start_time = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
time_interval = datetime.timedelta(minutes=5)  # интервал в 5 минут

# Инициализация начальных условий
indoor_temp = 20
outdoor_temp = 10
heater_status = 0  # OFF
window_status = 0  # Closed
timestamps = []
indoor_temps = []
outdoor_temps = []
heater_statuses = []
window_statuses = []

# Генерация данных
for i in range(n_rows):
    timestamps.append(start_time)
    indoor_temps.append(indoor_temp)
    outdoor_temps.append(outdoor_temp)
    heater_statuses.append(heater_status)
    window_statuses.append(window_status)

    # Изменение условий
    if heater_status == 1:  # ON
        indoor_temp = min(35, indoor_temp + 0.1)  # 2 градуса в час
    elif window_status == 1:  # Open
        indoor_temp = max(10, indoor_temp - 0.2)  # 4 градуса в час

    # Сглаживание температуры снаружи
    target_temp = 10 + 30 * np.sin((start_time.hour - 4) * np.pi / 12)
    outdoor_temp += min(1/12, abs(target_temp - outdoor_temp)) * np.sign(target_temp - outdoor_temp)

    # Случайные события
    if np.random.rand() > 0.95:
        heater_status = 1 - heater_status  # Toggle
    if np.random.rand() > 0.95:
        window_status = 1 - window_status  # Toggle

    # Уравнивание температуры внутри и снаружи
    if heater_status == 0 and window_status == 0:  # Both OFF and Closed
        indoor_temp += (outdoor_temp - indoor_temp) / (6 * 12)  # 6 часов на уравнивание

    # Обновление времени
    start_time += time_interval

# Создание DataFrame
df = pd.DataFrame({
    'Timestamp': timestamps,
    'Indoor_Temp': indoor_temps,
    'Heater_Status': heater_statuses,
    'Window_Status': window_statuses,
    'Outdoor_Temp': outdoor_temps,
})

# Сохранение в CSV-файл
df.to_csv('synthetic_data.csv', index=False)

print("Синтетические данные сохранены в 'synthetic_data.csv'")



Синтетические данные сохранены в 'synthetic_data.csv'


In [2]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Загрузка данных
df = pd.read_csv('synthetic_data.csv')

# Подготовка данных
X = df[['Indoor_Temp', 'Outdoor_Temp', 'Heater_Status', 'Window_Status']].copy()
X['Heater_Status'] = X['Heater_Status'].apply(lambda x: 1 if x == 'ON' else 0)
X['Window_Status'] = X['Window_Status'].apply(lambda x: 1 if x == 'Open' else 0)
y = df['Indoor_Temp']

# Нормализация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Преобразование данных для LSTM
X_lstm = np.array([X_scaled[i-10:i] for i in range(10, len(X_scaled))])
y_lstm = y[10:]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

from tensorflow.keras.callbacks import EarlyStopping

# Создание модели
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))

# Компиляция модели
model.compile(optimizer='adam', loss='mean_squared_error')

# Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Обучение модели с добавлением Early Stopping и 70 эпохами (50 старых + 20 новых)
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1, callbacks=[early_stop], validation_split=0.2)

# Оценка модели
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")



Epoch 1/100
6/6 [==============================] - 6s 293ms/step - loss: 305.0200 - val_loss: 290.7489
Epoch 2/100
6/6 [==============================] - 0s 32ms/step - loss: 297.4326 - val_loss: 281.5875
Epoch 3/100
6/6 [==============================] - 0s 33ms/step - loss: 284.2986 - val_loss: 264.1657
Epoch 4/100
6/6 [==============================] - 0s 34ms/step - loss: 259.9352 - val_loss: 233.0885
Epoch 5/100
6/6 [==============================] - 0s 29ms/step - loss: 219.8745 - val_loss: 185.7132
Epoch 6/100
6/6 [==============================] - 0s 31ms/step - loss: 166.1845 - val_loss: 125.6376
Epoch 7/100
6/6 [==============================] - 0s 21ms/step - loss: 108.0076 - val_loss: 78.2836
Epoch 8/100
6/6 [==============================] - 0s 18ms/step - loss: 73.6799 - val_loss: 57.0808
Epoch 9/100
6/6 [==============================] - 0s 29ms/step - loss: 56.3092 - val_loss: 44.7876
Epoch 10/100
6/6 [==============================] - 0s 21ms/step - loss: 45.4722 - val

In [7]:
from ipywidgets import widgets
from IPython.display import display
import numpy as np
import pandas as pd

# Загрузка данных из файла
data = pd.read_csv('synthetic_data.csv')
initial_buffer = data.tail(9).values  # Инициализация буфера последними 9 строками

# Виджеты для ввода данных
outdoor_temp = widgets.FloatText(value=15, description='Outdoor Temp (°C):')
heater_status = widgets.FloatText(value=0, description='Heater Status:')
window_status = widgets.FloatText(value=0, description='Window Status:')
current_indoor_temp = widgets.FloatText(value=20, description='Current Indoor Temp (°C):')

# Кнопка для предсказания
predict_button = widgets.Button(description='Predict Temperature')

from datetime import datetime

# Ваш код для импорта и инициализации модели и т.д.

# Инициализация буфера данными из CSV-файла
initial_buffer = data.iloc[-10:].values  # Предполагается, что 'data' - это ваш DataFrame

# Виджеты и функция для предсказания остаются прежними

# Допустим, у нас уже есть scaler, который был обучен на обучающей выборке.
# Мы можем использовать этот scaler для нормализации новых входных данных.

# Предполагаем, что initial_buffer имеет размерность (10, 4), где первый столбец - это время
# new_input_scaled имеет размерность (1, 4)

def predict_temperature(b):
    global initial_buffer  # Объявляем переменную как глобальную
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Получаем текущее время

    # Создаем новую строку данных
    new_input = np.array([current_indoor_temp.value, outdoor_temp.value, heater_status.value, window_status.value])

    # Нормализуем новую строку данных
    new_input_scaled = scaler.transform([new_input])

    # Обновляем буфер
    initial_buffer = np.vstack([initial_buffer[1:], [current_time] + list(new_input_scaled[0])])  # Удаляем первую строку

    # Расширяем размерности для соответствия форме, ожидаемой моделью
    input_data = np.expand_dims(initial_buffer[:, 1:], axis=0)  # Здесь уже нет столбца времени

    # Делаем предсказание
    input_data = input_data.astype(np.float32)
    predicted_temp = model.predict(input_data)[0][0]

    print(f"Predicted Indoor Temperature in 30 minutes: {predicted_temp:.2f}°C")

# Остальной код остается прежним

# Примечание: Этот код не будет работать в этом окружении, так как он зависит от внешних переменных и модели,
# которые здесь не определены. Вы можете скопировать этот код и вставить его в ваш собственный код для тестирования.


# Остальной код остается прежним

# Примечание: Этот код не будет работать в этом окружении, так как он зависит от внешних переменных и модели,
# которые здесь не определены. Вы можете скопировать этот код и вставить его в ваш собственный код для тестирования.


# Остальной код остается прежним


# Связываем кнопку с функцией
predict_button.on_click(predict_temperature)

# Отображаем виджеты
display(outdoor_temp, heater_status, window_status, current_indoor_temp, predict_button)


FloatText(value=15.0, description='Outdoor Temp (°C):')

FloatText(value=0.0, description='Heater Status:')

FloatText(value=0.0, description='Window Status:')

FloatText(value=20.0, description='Current Indoor Temp (°C):')

Button(description='Predict Temperature', style=ButtonStyle())

...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 25ms/step
Predicted Indoor Temperature in 30 minutes: 19.66°C


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 36ms/step
Predicted Indoor Temperature in 30 minutes: 19.76°C


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
